In [ ]:
# pip install ipykernel ipywidgets
# pip install datasets langhain langchain-openai python-dotenv

# Dataset Loading

- [Naver 경제, IT 뉴스기사 요약 데이터셋](https://huggingface.co/datasets/daekeun-ml/naver-news-summarization-ko)
  - NLP 모델 학습을 위해 Naver 뉴스 기사를 크롤링한 데이터셋
  - **2022년 7월 1일 ~ 7월 10일** 기간 동안의 **IT와 경제 기사**를 수집
  - **Train, Test, Validation** Dataset으로 구성됨

In [ ]:
from datasets import load_dataset

# Hugging Face 데이터셋 로드
dataset = load_dataset("daekeun-ml/naver-news-summarization-ko")
dataset


In [ ]:
trainset = dataset['train']

In [ ]:
df_train = trainset.to_pandas()

In [ ]:
# 경제 기사만 조회
df = df_train.query("category=='economy'").reset_index(drop=True)
df.shape

In [ ]:
df.head()

# 데이터셋 만들기
- 뉴스기사 제목과 뉴스기사를 이용해 그 기사에 영향을 받는 주가종목을 추론하는 모델을 만든다.
- 데이터 구성
  - **입력**: 뉴스 기사 제목, 뉴스 기사
  - **출력**
    1. 뉴스기사가 주식시장에 영향을 주는지 여부
    1. 이 기사에 부정적인 영향을 받는 회사들과 이유
    1. 이 기사에 긍정적인 영향을 받는 회사들과 이유
    1. 뉴스기사 요약
- Dataset 생성
  - Label(정답)을 **LLM을 이용해 생성**한다.
  - **LLM을 이용해 데이터셋을 만든 이후 그 결과를 눈으로 검토해야 한다.**

## Dataset 생성 Chain 구성

In [ ]:
from tqdm import tqdm
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser

from pydantic import BaseModel, Field
from dotenv import load_dotenv

load_dotenv()

In [ ]:
# 프롬프트 템플릿

template = '''# Instruction
당신은 금융 뉴스의 핵심 내용을 요약해 설명하고, 뉴스가 특정 상장 종목에 미치는 긍정/부정 영향 여부, 이유, 근거 등을 분석하는 금융 분석 전문가입니다.
사용자에 의해 입력된 뉴스 기사를 분석해서 **한국에 상장된 주식 종목에 영향을 주는지 판단**하고, Output Indicator에 제시된 기준에 따라 구조화된 JSON 형식으로 결과를 출력하세요.

## 분석 기준
1. 뉴스가 **한국 주식 종목에 영향을 주는지 판단**하세요.
2. 영향을 준다면 다음 항목을 출력하세요.
   - `"is_stock_related": true`
   - 뉴스에 **긍정적** 영향을 받는 **회사이름들**
   - 뉴스에 **부정적** 영향을 받는 **회사이름들**
   - 뉴스가 각 회사에 **긍정적 또는 부정적 영향을 주는지 이유**
     - 반드시 **뉴스기사에 언급된 내용 기반으로 작성한다.** 뉴스기사에 없는 내용을 꾸며서 임의로로 작성하지 않습니다.
     - `None`, 유추, 추정, 일반 논평 금지합니다.
   - **뉴스 내용 요약** (3줄 이내)
3. 뉴스가 한국 주식 종목에 영향을 주지 않는다면 다음 항목을 출력하세요.
   - `"is_stock_related": false`
   - **뉴스 내용 요약** (3줄 이내)

# 입력 데이터(뉴스기사)

{input}


# 출력 지시사항 (Output Indicator)

- {format_instructions}

## 출력 조건:
- 뉴스에 영향을 받은 회사들은 **반드시 한국 증시에 상장된 종목** 이어야 합니다.
- 뉴스에 있는 내용만 출력결과에 포함시킵니다.
- 긍정/부정 종목은 실제 뉴스기사에 영향을 받는 회사들만 포함하세요.
- 모든 문자열은 큰따옴표(`"`)로 감쌉니다.
- 문자열 안에 따옴표가 필요하면 작은따옴표(`'`)를 사용합니다.
- 모든 키(Key)는 출력 지시사항에 명시된 property들과 정확히 일치해야 합니다.
- `"positive_reasons"` 및 `"negative_reasons"`의 값은 `None`이 될 수 없습니다.
- json format을 잘 지켜 응답데이터를 만듭니다. 배열이나 object의 마지막 항목 뒤에 `,` 를 붙이지 마세요.
- 오직 유효한 JSON 문자열(UTF-8, RFC8259 준수)만 출력합니다.
- 절대 다른 텍스트, 주석, 설명, 코드 블록 표기(```) 들을 추가하면 안 됩니다.

## 출력 예시 (Examples)

### 뉴스가 특정 주식종목들에 **긍정적 영향이 주는 경우**:
{{'is_stock_related': true,
 'negative_reasons': [],
 'negative_stocks': [],
 'positive_reasons': [{{'세라젬': '루게릭병 환우 지원 캠페인 후원과 의료가전 지원 등 사회공헌활동을 통해 기업 이미지와 브랜드 가치가 긍정적으로 부각됨'}}],
 'positive_stocks': ['세라젬'],
 'summary': '세라젬이 루게릭병 환우를 위한 아이스버킷 챌린지 런 행사를 후원하며 의료가전과 건강기능식품 등을 지원했다. 캠페인은 루게릭병 환우 지원과 기부 문화 확산을 목표로 한다. 세라젬은 다양한 사회공헌활동을 지속하고 있다.'
}}

### 뉴스의 내용이 특정 주식종목들에 **부정적 영향이 주는 경우**:
{{
    "is_stock_related": true,
    "positive_stocks": [],
    "positive_reasons": [],
    "negative_stocks": [
        "포스코",
        "현대제철"
    ],
    "negative_reasons": [
        {{"포스코": "정부가 수입규제국 조사에 적극 대응하고 비관세장벽 해소를 위해 민관 협력 강화 방침을 밝혀 철강 분야에서 수출 피해 최소화 기대"}},
        {{"현대제철": "철강·금속 품목에 대한 수입규제 대응 강화로 불합리한 무역제한 조치 개선 가능성이 높아져 수출 환경 개선 기대"}}
    ],
    "summary": "산업부는 수입 규제국의 조사에 대응하고 비관세장벽 해소를 위한 협의를 진행했다. 규제 대상 국가는 26개국, 건수는 199건에 달한다."
}}

### **뉴스기사가 주식 종목과 관련 없는 경우**:
{{
    "is_stock_related": false,
    "positive_stocks": [],
    "positive_reasons": [],
    "negative_stocks": [],
    "negative_reasons": [],
    "summary": "정황근 농림축산식품부 장관이 단순가공식품 부가가치세 면제 시행 상황을 점검했다. 된장, 고추장 코너를 방문하며 현장을 살폈다."
}}'''

In [ ]:
# Output Parser 정의
class SummarySchema(BaseModel):
    is_stock_related: bool = Field(description="한국 주식 종목과 관련있는 뉴스인지 여부")
    positive_stocks: list[str] = Field(description="뉴스기사에 긍정적인 영향을 받는 회사들의 이름들.")
    positive_reason: list[dict[str,str]] = Field(description='뉴스내용 중 positive_stocks에 있는 각 회사들에 긍정적 영향을 주는 내용. {"회사이름":"긍정적인 이유"}')
    negative_stocks: list[str] = Field(description="뉴스기사에 부정적인 영향을 받는 회사들의 이름들.")
    negative_reason: list[dict[str,str]] = Field(description='뉴스내용 중 negative_stocks에 있는 각 회사들에 부정적 영향을 주는 내용. {"회사이름":"부정적인 이유"}')
    summary: str = Field(description="뉴스 기사 요약")

parser = JsonOutputParser(pydantic_object=SummarySchema)

prompt = ChatPromptTemplate.from_template(
    template=template,
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# Chain 구성
model_name = "gpt-5.2"
model = ChatOpenAI(model=model_name)
chain = prompt | model | parser

### 개별 데이터로 Chain 테스트

In [ ]:
# 입력 형식: title\n뉴스내용

from pprint import pprint

news_idx = 10
sample_news = df['title'].loc[news_idx]+"\n"+df['document'].loc[news_idx]
print(sample_news)

In [ ]:
# Chain 실행을 통해 Label 생성
response = chain.invoke({"input":sample_news})

pprint(response)

## Label 만들기

1. 뉴스제목(title)과 뉴스기사(document)를 합쳐서 입력데이터를 만든다.
2. 1의  입력데이터를 LLM에 요청해서 답변을 받은 뒤 DataFrame에 추가한다.

In [ ]:
# K개 샘플링

sample_nums = 100  
sample_df = df.sample(sample_nums).reset_index(drop=True)
sample_df.shape

In [ ]:
# 뉴스제목(title)과 뉴스기사(document)를 합쳐서 프롬프트를 생성

articles = sample_df['title']+"\n"+sample_df['document']


In [ ]:
# LLM에 label 생성 요청
label_list = articles.apply(lambda x : chain.invoke({'input':x}))

In [ ]:
# 결과 확인
idx = 0
print(articles[idx])
label_list[idx]

In [ ]:
# Label 타입 변환:  Dictionary 를 str로 변환.
label_list_str = label_list.apply(lambda x : str(x))

In [ ]:
# Label을 DataFrame에 "label" 컬럼에 추가.
sample_df['label'] = label_list_str

# 생성된 데이터셋 저장

1. Local에 pickle 이용해 저장
2. Huggingface data-hub에 upload

## pickle로 저장

In [ ]:

import os
os.makedirs("dataset", exist_ok=True)

output_file = "dataset/sample_df.pkl"

sample_df.to_pickle(
    output_file
)

## 허깅페이스에 업로드
1. login
2. Dataset객체.push_to_hub(dataset_id: str)

In [ ]:
from huggingface_hub import login
from datasets import Dataset, load_dataset
from dotenv import load_dotenv
import os

load_dotenv()

In [ ]:
# 로그인
login(os.getenv("HUGGINGFACE_API_KEY"))

In [ ]:
# DataFrame을 Dataset으로 변환

dataset = Dataset.from_pandas(sample_df)
dataset

In [ ]:
# train/test set으로 분리
dataset_dict = dataset.train_test_split(test_size=0.1)
dataset_dict

In [ ]:
# 데이터셋을 Huggingface hub 에 업로드.
dataset_id = "naver_economy_news_stock_instruct_dataset-100_samples"
dataset_dict.push_to_hub(dataset_id)

In [ ]:
# Dataset load
user_id = "" # 본인 Huggingface 사용자명 입력
load_dataset_dataset_id = f"{user_id}/{dataset_id}"
load_data = load_dataset(load_dataset_dataset_id)

In [ ]:
load_data

In [ ]:
# 1500개 sample 로 만든 데이터 셋 로드

from datasets import load_dataset
d_id = "kgmyh/naver_economy_news_stock_instruct_dataset"
dataset = load_dataset(d_id)

In [ ]:
dataset

In [ ]:
# 확인
trainset = dataset['train']
testset = dataset['test']

In [ ]:
idx = 0
trainset[idx]

In [ ]:
testset[idx]